# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 05:28:40] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=34395, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=35776947, constrained_json_whitespace_pattern=None, watchdog_timeout=30

[2025-06-28 05:28:40] Inferred chat template from model path: llama-2


[2025-06-28 05:28:51] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 05:28:51] Init torch distributed begin.


[2025-06-28 05:28:52] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 05:28:53] Load weight begin. avail mem=61.98 GB


...

[2025-06-28 05:28:57] Init torch distributed begin.
[2025-06-28 05:28:57] Init torch distributed ends. mem usage=0.00 GB
[2025-06-28 05:28:57] Load weight begin. avail mem=38.65 GB
[2025-06-28 05:28:57] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.20s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.20s/it]

[2025-06-28 05:28:58] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-06-28 05:28:58] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-28 05:28:58] Memory pool end. avail mem=37.40 GB


[2025-06-28 05:28:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB
[2025-06-28 05:28:59] INFO:     Started server process [1163974]
[2025-06-28 05:28:59] INFO:     Waiting for application startup.
[2025-06-28 05:28:59] INFO:     Application startup complete.
[2025-06-28 05:28:59] INFO:     Uvicorn running on http://127.0.0.1:34395 (Press CTRL+C to quit)


[2025-06-28 05:28:59] INFO:     127.0.0.1:50306 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 05:29:00] INFO:     127.0.0.1:50318 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 05:29:00] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-28 05:30:03] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-28 05:30:32] INFO:     127.0.0.1:50330 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 05:30:32] The server is fired up and ready to roll!


[2025-06-28 05:30:35] INFO:     127.0.0.1:50334 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-28 05:30:35] Child process unexpectedly failed with exitcode=9. pid=1165052
[2025-06-28 05:30:35] Child process unexpectedly failed with exitcode=9. pid=1164571


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 05:30:46] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=32920, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=463055621, constrained_json_whitespace_pattern=None, watchdog_timeout=300

[2025-06-28 05:30:46] Inferred chat template from model path: llama-2


[2025-06-28 05:30:58] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 05:30:58] Init torch distributed begin.


[2025-06-28 05:30:58] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 05:30:59] Load weight begin. avail mem=30.29 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.32s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.32s/it]

[2025-06-28 05:31:05] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=30.74 GB, mem usage=-29.47 GB.
[2025-06-28 05:31:05] max_total_tokens=20480 is larger than the profiled value 17835. Use the profiled value instead.
[2025-06-28 05:31:05] KV Cache is allocated. #tokens: 17835, K size: 0.14 GB, V size: 0.14 GB
[2025-06-28 05:31:05] Memory pool end. avail mem=30.46 GB


[2025-06-28 05:31:05] max_total_num_tokens=17835, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=47.84 GB
[2025-06-28 05:31:05] INFO:     Started server process [1171549]
[2025-06-28 05:31:05] INFO:     Waiting for application startup.
[2025-06-28 05:31:05] INFO:     Application startup complete.
[2025-06-28 05:31:05] INFO:     Uvicorn running on http://127.0.0.1:32920 (Press CTRL+C to quit)


[2025-06-28 05:31:06] INFO:     127.0.0.1:42550 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 05:31:06] INFO:     127.0.0.1:42552 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 05:31:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:31:09] INFO:     127.0.0.1:42566 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 05:31:09] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-28 05:31:11] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:31:11] INFO:     127.0.0.1:49736 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-28 05:31:12] Child process unexpectedly failed with exitcode=9. pid=1171848
[2025-06-28 05:31:12] Child process unexpectedly failed with exitcode=9. pid=1171714


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 05:31:24] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=34118, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=806593691, constrained_json_whitespace_pattern=None,

[2025-06-28 05:31:36] Casting torch.bfloat16 to torch.float16.


[2025-06-28 05:31:36] Casting torch.bfloat16 to torch.float16.
[2025-06-28 05:31:36] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 05:31:36] Init torch distributed begin.


[2025-06-28 05:31:37] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 05:31:38] Load weight begin. avail mem=63.56 GB


...

[2025-06-28 05:31:54] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-06-28 05:31:54] INFO:     Started server process [1173275]
[2025-06-28 05:31:54] INFO:     Waiting for application startup.
[2025-06-28 05:31:54] INFO:     Application startup complete.
[2025-06-28 05:31:54] INFO:     Uvicorn running on http://127.0.0.1:34118 (Press CTRL+C to quit)


[2025-06-28 05:31:55] INFO:     127.0.0.1:58710 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 05:31:55] INFO:     127.0.0.1:58716 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 05:31:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:31:58] INFO:     127.0.0.1:58720 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 05:31:58] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-28 05:32:00] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:32:00] INFO:     127.0.0.1:47246 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-28 05:32:00] Child process unexpectedly failed with exitcode=9. pid=1174349
[2025-06-28 05:32:00] Child process unexpectedly failed with exitcode=9. pid=1173927


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 05:32:14] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=33101, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=968491215, constrained_json_whitespace_pattern=None, watchdog

[2025-06-28 05:32:25] Casting torch.bfloat16 to torch.float16.


[2025-06-28 05:32:25] Casting torch.bfloat16 to torch.float16.
[2025-06-28 05:32:25] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 05:32:25] Init torch distributed begin.


[2025-06-28 05:32:26] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 05:32:27] Load weight begin. avail mem=60.16 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.26it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.26it/s]

[2025-06-28 05:32:41] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=58.29 GB, mem usage=1.82 GB.
[2025-06-28 05:32:41] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-06-28 05:32:41] Memory pool end. avail mem=58.21 GB


[2025-06-28 05:32:41] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.89 GB


[2025-06-28 05:32:42] INFO:     Started server process [1176140]
[2025-06-28 05:32:42] INFO:     Waiting for application startup.
[2025-06-28 05:32:42] INFO:     Application startup complete.
[2025-06-28 05:32:42] INFO:     Uvicorn running on http://127.0.0.1:33101 (Press CTRL+C to quit)


[2025-06-28 05:32:42] INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-28 05:32:43] INFO:     127.0.0.1:50742 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 05:32:43] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:32:45] INFO:     127.0.0.1:50756 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 05:32:45] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-28 05:32:48] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:32:48] INFO:     127.0.0.1:50766 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-28 05:32:48] Child process unexpectedly failed with exitcode=9. pid=1176961
[2025-06-28 05:32:48] Child process unexpectedly failed with exitcode=9. pid=1176666
[2025-06-28 05:32:48] Child process unexpectedly failed with exitcode=9. pid=1176963


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 05:33:01] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=31251, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=187155328, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downlo

[2025-06-28 05:33:12] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 05:33:12] Init torch distributed begin.


[2025-06-28 05:33:13] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 05:33:14] Load weight begin. avail mem=78.50 GB


[2025-06-28 05:33:14] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  8.30it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.87it/s]

[2025-06-28 05:33:18] Load weight end. type=MiMoMTP, dtype=torch.bfloat16, avail mem=57.83 GB, mem usage=2.71 GB.
[2025-06-28 05:33:18] KV Cache is allocated. #tokens: 20480, K size: 1.41 GB, V size: 1.41 GB
[2025-06-28 05:33:18] Memory pool end. avail mem=54.88 GB


[2025-06-28 05:33:19] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-28 05:33:19] INFO:     Started server process [1177735]
[2025-06-28 05:33:19] INFO:     Waiting for application startup.
[2025-06-28 05:33:19] INFO:     Application startup complete.
[2025-06-28 05:33:19] INFO:     Uvicorn running on http://0.0.0.0:31251 (Press CTRL+C to quit)


[2025-06-28 05:33:20] INFO:     127.0.0.1:43658 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-28 05:33:20] INFO:     127.0.0.1:43666 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 05:33:20] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:33:21] INFO:     127.0.0.1:43668 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 05:33:21] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-28 05:33:25] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 05:33:26] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, accept len: 1.71, cuda graph: False, gen throughput (token/s): 10.03, #queue-req: 0


[2025-06-28 05:33:26] Decode batch. #running-req: 1, #token: 160, token usage: 0.01, accept len: 1.77, cuda graph: False, gen throughput (token/s): 110.20, #queue-req: 0


[2025-06-28 05:33:27] Decode batch. #running-req: 1, #token: 228, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 105.05, #queue-req: 0


[2025-06-28 05:33:28] Decode batch. #running-req: 1, #token: 297, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 107.89, #queue-req: 0


[2025-06-28 05:33:28] INFO:     127.0.0.1:43674 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-28 05:33:28] Child process unexpectedly failed with exitcode=9. pid=1178019
[2025-06-28 05:33:28] Child process unexpectedly failed with exitcode=9. pid=1177884


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).